In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from frozenlake_helper import get_frozenlake_env, policy_1, policy_2, policy_3, policy_4
import numpy as np

In [3]:
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

In [4]:
policies = [policy_1, policy_2, policy_3, policy_4]

# Instancio entorno

In [5]:
env = get_frozenlake_env(is_slippery=False, step_penalty=0)

### Algunas pruebas para familiarizarse con los entornos de openAI

In [6]:
start_position = 1
env.reset(start_position)
env.render()

# Genéro acción y recibo siguiente estado, recompenza y si terminó el episodio
obs, reward, done, info = env.step(DOWN)
print()
print(obs, reward, done, info)
print()
env.render()


SFFF
FFFF
HFFG

5 0 False {'prob': 1.0}

  (Down)
SFFF
FFFF
HFFG


In [7]:
env.step(RIGHT)
print(obs, reward, done, info)
env.render()

5 0 False {'prob': 1.0}
  (Right)
SFFF
FFFF
HFFG


# Armar una función que corra un episodio completo y devuelta el retorno acumulado

Recibe el entorno, la política y la posición inicial del robot
Devuelve el retorno (suma de los rewards)

In [8]:
def run_episode(env, policy, start_pos, gamma=1.0):
    obs = env.reset(start_pos)
    done = False
    total_return = 0
    while not done:
        # Implementar
        obs, reward, done, info = env.step(policy[obs])
        total_return = total_return + reward
        # break
    
    return total_return

# Ejercicio 1:
- Entorno determinístico
- Penalidad del paso = 0 
- Sin discount
- Partiendo de posición 0: (0,0)

In [9]:
start_pos = 0
step_penalty = 0

In [10]:
env = get_frozenlake_env(False, step_penalty=step_penalty)

### Evaluar las distintas políticas pariendo desde la posición 0

In [11]:
for i, policy in enumerate(policies):
    r = run_episode(env, policy=policy, start_pos=start_pos)
    print(f'Return policy_{i+1}:', r)

Return policy_1: 1.0
Return policy_2: 1.0
Return policy_3: 1.0
Return policy_4: 0


# Ejercicio 2:
- Entorno determinístico
- Penalidad del paso = 0.01 
- Sin discount
- Partiendo de posición 0: (0,0)

In [12]:
start_pos = 0
step_penalty = 0.01
env = get_frozenlake_env(False, step_penalty=step_penalty)

In [13]:
for i, policy in enumerate(policies):
    r = run_episode(env, policy=policy, start_pos=start_pos)
    print(f'Return policy_{i+1}:', r)

Return policy_1: 0.96
Return policy_2: 0.96
Return policy_3: 0.92
Return policy_4: -0.02


# Ejercicio 3:
- Entorno determinístico
- Penalidad del paso = 0 
- gamma = 0.9
- Partiendo de posición 0: (0,0)

### En este entorne debe crear la función get_discounted_return y modificar run_episode acorde

In [61]:
def get_discounted_return(r, gamma=0.9):
    # r es una lista o un numpy.array con todos los retornos del episodio
    r = np.array(r, dtype=float)
    # Implementar
    discounted_r = 0
    for x in range(0,len(r)):
        discounted_r = discounted_r + r[x] * gamma ** x
    #print(r)
    #discounted_r = 0 
    return discounted_r

In [62]:
def run_episode(env, policy, start_pos, gamma=1.0):
    obs = env.reset(start_pos)
    done = False
    rewards = []
    while not done:
        action = policy[obs]
        obs, reward, done, info = env.step(action)
        rewards.append(reward)
    return get_discounted_return(rewards, gamma)

In [63]:
start_pos = 0
step_penalty = 0
gamma = 0.9
env = get_frozenlake_env(False, step_penalty=step_penalty)

In [64]:
for i, policy in enumerate(policies):
    r = run_episode(env, policy=policy, start_pos=start_pos, gamma=gamma)
    print(f'Return policy_{i+1}:', r)

Return policy_1: 0.6561
Return policy_2: 0.6561
Return policy_3: 0.4304672100000001
Return policy_4: 0.0


# Ejercicio 4:
- Entorno aleatorio
- Penalidad del paso = 0 
- gamma = 1.0 (sin discount)
- Partiendo de posición 0: (0,0)

In [65]:
start_pos = 0
step_penalty = 0
gamma = 1.0
env = get_frozenlake_env(True, step_penalty=step_penalty)

### En este caso cuando se decide una acción, el agente se moverá hacia el lugar indicado con una probabilidad de 0.33, y se moverá hacia cualquiera de los costados con probabilidad 0.33

### Cada vez que corra un episodio obtendrá un resultado diferente, intentelo

In [66]:
# Correr varias veces y ver que el resultado cambia con cada iteración
for i, policy in enumerate(policies):
    r = run_episode(env, policy=policy, start_pos=start_pos, gamma=gamma)
    print(f'Return policy_{i+1}:', r)

Return policy_1: 1.0
Return policy_2: 1.0
Return policy_3: 1.0
Return policy_4: 0.0


### Armar una función que corra el episodio N veces y devuleva los retornos, la media y desvio

In [67]:
def get_expected_return(env, policy, N=5000, start_pos=0, gamma=1.0):        
    rewards = []
    
    for i in range(N):
        # Implementar
        r = run_episode(env, policy, start_pos, gamma)
        rewards.append(r)
        #pass

    
    
    return rewards, np.mean(rewards), np.std(rewards)

In [68]:
for i, policy in enumerate(policies):
    r = get_expected_return(env, policy, N=10_000, start_pos=start_pos, gamma=gamma)
    print(f'Return policy_{i+1}:', r[1], r[2])

Return policy_1: 0.8013 0.39902169113971736
Return policy_2: 0.5404 0.4983651673221152
Return policy_3: 0.5772 0.49400421050837207
Return policy_4: 0.0 0.0


In [22]:
for i, policy in enumerate(policies):
    r = get_expected_return(env, policy, N=10_000, start_pos=start_pos, gamma=gamma)
    print(f'Return policy_{i+1}:', r[1], r[2])

[0. 0.]
[0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0.]
[0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0.

# Ejercicio 5:
- Entorno determinístico
- Penalidad del paso = 0 
- gamma = 0.9 (sin discount)
- Partiendo de posición 0: (0,0)

### Calcular la value-function de todas los politicas

In [23]:
step_penalty = 0
gamma = 0.9
env = get_frozenlake_env(False, step_penalty=step_penalty)

In [24]:
for i, policy in enumerate(policies):
    print(f'Return policy_{i+1}:', r)
    for start_pos in range(12):
        if start_pos in policy:
            r = run_episode(env, policy=policy, start_pos=start_pos, gamma=gamma)
            print(start_pos, r)
    print()

Return policy_1: ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0